# Kubernetes Basics

> Open Source platform managing __containerized__ workloads __focused on declarative configuration and automation__

What Kubernetes allows us to do:
- Container deployment across clusters:
    - Which and how many containers should be deployed
    - How many replicas for fault tolerance
    - __Self-healing__ - if a container is down (not responding to health checks) __spin a new one__
    - __Auto-scaling__ - given increased/decreased requirements __spin up or destroy containers in the cluster__
- __Discoverability__ - each container can be accessed via it's name (or IP) and port
- __Load Balancing__ - traffic to the cluster might be distributed if one node is overloaded
- Automated rollouts and rollbacks - describe __the desired state__ and `k8s` tries to match it automatically
- Storage orchestration - allows us to mount storage (per-node, shared and others) to save/read data

> __Kubernetes is a "low level" platform, A TOP OF WHICH other projects reside; consider it "a programming language for deployment"__ 

## Containers == game changer

Let's see how containers changed the deployment landscape:

![](./images/container_evolution.svg)

__Benefits__:
- More lightweight than VMs
- Immutable
- Easy to create/destroy when needed
- __Developers can pack the code with all necessities__; ops take care of deploying "described black boxes" 
- Reproducible - runs the same everywhere (more than a VM)
- __Micro-services__ - app can be broken into multiple separate containers communicating with each other:
    - We only change image we need
    - No need to deploy everything a-new
    - Easily switchable components 

## Imperative vs Declarative

> Using imperative programming (or configuration) __we define each necessary step to get to the result__

Some examples could be:
- Create variable `x` and assign value `1` to it
- Scrape data from the website and get appropriate `<tag>`
- Pass data `X` through linear layer and apply sigmoid on top of it

In contrast, __declarative programming__:

> Using declarative programming (or configuration) __we describe the desired state of system without describing the steps__

Respectively for the examples above:
- I want variable `x` with value `1` assigned to it
- I want `<tag>` out of this website
- I want binary probabilities prediction for data `X`

> #### Kubernetes uses declarative programming to describe state of the whole system

# Kubernetes Components

> Kubernetes has __a lot__ of concepts one should be familiar with in order to work with the platform efficiently

When we __deploy Kubernetes__ we obtain a cluster with the components shown in the diagram below:

![](./images/components-of-kubernetes.svg)

## High level components overview

- __`Node`s__ - worker machines (__either physical or virtual__) which host containerized application
- __`POD`s__ - component of the application, __smallest deployable unit of work in `k8s`__:
    - think of it as our program running within `Node`
    - consists of a few containers which create applicaiton together
    - is bundled with `storage`, `network` address etc. to create a coherent deployment unit
- __Control Plane__ - manages `node`s and `POD`s

> For learning (at the beginning) we will have a single `Node` on the same physical machine

> In production environment __`Control Plane`__ can span multiple machines for increased fault tolerance

> In production environment we usually have multiple `Node`s (up to thousands)

## Control Plane Components

### kube-apiserver

> __Exposes `Kubernetes` API using which we can communicate with the cluster__

Communication with the cluster can be done via:
- `http` requests as `kube-apiserver` provides REST server (__read more about it [here](https://kubernetes.io/docs/concepts/overview/kubernetes-api/)__)
- __through command line__:
    - `kubectl` - control kubernetes cluster and workload
    - `kubeadm` - bootstraps kubernetes clusters (initializing it from config, tearing down etc.)
      
> __We should almost exclusively use `command line` tools to interact with `kube-apiserver`!__

Reasons are:
- increased readability
- easier automation
- `cli` tools handle different internal API version of `k8s` for us

### etcd

> __`etcd` stores Kubernetes Objects which define the whole cluster__

Features:
- Data is stored locally on control plane node(s)
- __In case all of our control planes go down we lose cluster state__ (and we have to recreate the whole infrastructure a-new)

> `etcd` data should be backed in an external storage, check [here](https://kubernetes.io/docs/tasks/administer-cluster/configure-upgrade-etcd/#backing-up-an-etcd-cluster) for more information

### kube-scheduler

> __Watched newly created `POD`s and assigns them to `node`__

How is the node chosen?
- Resource requirements for our `POD`
- Applied policies/constraints
- Data locality (__move code towards data `spark`__)

If you are curious about scheduling PODs in more detail check out [this part of documentation](https://kubernetes.io/docs/concepts/scheduling-eviction/assign-pod-node/)

### kube-controller-manager

> Runs controller processes

What is a `control process`?

> Loop which watches the cluster (through `apiserver`) and __attempts to move the current state to the desired one__

Each controller runs in a separate process, one example could be a `node controller` which responds when node goes down.

### cloud-controller-manager

> Similar to `kube-controller-manager` __but watches state of cloud resources__

For example `node controller` queries cloud provider for node health check. 

## Node Components

Each node consists of the following:

### kubelet

> Given `PodSpec` (specification we will later see) __ensures appropriate containers run within `POD` and are healthy__

### kube-proxy

> Governs networks rules on nodes. __Enables communication to/from `POD`s within and outside cluster__

### Container runtime

> The container runtime is the software that is responsible for running containers

Some possibilities include:
- [`Docker`](https://docs.docker.com/engine/) (which we already know)
- [`containerd`](https://containerd.io/docs/) - simple container runtime (alternative to `Docker`)

Any project which fulfills [Kubernetes CRI specification](https://github.com/kubernetes/community/blob/master/contributors/devel/sig-node/container-runtime-interface.md) 

# Kubernetes Objects

> __Persistent entities in `k8s` system which represent OUR DESIRED STATE of cluster__

Using objects we can describe:
- What applications and where (which node) these are running
- How applications behave (e.g. when to restart, delete, upgrade)
- Resources available to applications

In order to create them __we have to request `kubelet-apiserver`'s API__!

As mentioned previously, instead of directly sending REST requests we can (and will) use `kubectl` for that.

> __To specify how we want `k8s` to run we have to describe objects__

> Each `k8s` object has a unique ID assigned by the platform

## Describing Kubernetes objects

As API is RESTful we can include `JSON` description of object.

__This way is supported, although hard to maintain__

> __What we can do instead is use more readable `YAML` to specify our objects and let `kubectl` transform the representation to `JSON`__

Let's see an example object configuration and describe the fields:

In [2]:
---
apiVersion: apps/v1
kind: Deployment
metadata:
  name: nginx-deployment
spec:
  selector:
    matchLabels:
      app: nginx
  replicas: 2 # tells deployment to run 2 pods matching the template
  template:
    metadata:
      labels:
        app: nginx
    spec:
      containers:
      - name: nginx
        image: nginx:1.14.2
        ports:
        - containerPort: 80

bash: ---: command not found
bash: apiVersion:: command not found
bash: kind:: command not found
bash: metadata:: command not found
bash: name:: command not found
bash: spec:: command not found
bash: selector:: command not found
bash: matchLabels:: command not found
bash: app:: command not found
bash: replicas:: command not found
bash: template:: command not found
bash: metadata:: command not found
bash: labels:: command not found
bash: app:: command not found
bash: spec:: command not found
bash: containers:: command not found
bash: -: command not found
bash: image:: command not found
bash: ports:: command not found
bash: -: command not found


: 1

- `apiVersion` - specifies `k8s` API version (`kubectl` will use it to make appropriate requests)
- `kind` - Kind of object we want to create (we will talk about it later during __Workloads__)
- `metadata` - Uniquely defines the object, usually using `name` field

## spec and... status

> __`spec` describes DESIRED characteristics of our resource__

This one is provided by user, and, in this case, tells the `kubernetes` platform that:
- Every POD which has a label `app` with name `nginx` is part of `Deployment` kind (__described later__)
- Two `POD`s should be run on the cluster (`replicas`)
- Each one should be created from a `template` which:
    - Gives `app` label with a value `nginx`
    - `spec`ifies it is created from a single container which will be named `nginx` (for discoverability)
    - Uses `image` `nginx:1.14.2`
    - And has container port `80` open for communication
    
Okay, where is `status` though?

> __`status` describes CURRENT state of the resource__

__While we provide `spec`, `status` is updated internally by `kubernetes`__

Providing `status` has the following benefits:
- Allows users to query the current status of the deployment
- __Allows controllers to act based on the current `status`__ and drive the objects toward the desired state

### Desired vs Actual state

A little digression:

> __Do not worry about `actual` state, because our claster might never reach it !__

As long as `controlers` are continuously running everything is fine.

Descrepancy beetwen current and desired state might happen due to plethora of reasons, e.g: 

- Freaquent updates of `config` files (`kubernetes` had no time to act accordingly to our changes)
- We are handling hundreds of nodes and our change hasn't being propagate yet
- Random node failures (`kubernetes` will reinstantiate necessary `POD`s) 

## Creating and managing objects

Now that we have our object described we should communicate to `kube-apiserver` we want it created.

There are `3` techniques to do that:
- __Imperative commands__ - operates on existing objects __via commands__ (discouraged!)
- __Imperative object configuration__ - operate on individual files
- __Declarative object configuration__ - works on directories of files

> __Approaches should not be mixed together as it may result in `undefined behavior`!__

__We will mostly focus on `declarative` approach__

### Imperative object configuration

> __`kubectl` specifies operation to carry given a `.yaml` config file__

Some examples:

> Create the objects defined in a configuration file:

In [ ]:
kubectl create -f nginx.yaml

> Delete the objects defined in two configuration files:

In [ ]:
kubectl delete -f nginx.yaml -f redis.yaml

> Update the objects defined in a configuration file by overwriting the live configuration:

In [ ]:
kubectl replace -f nginx.yaml

### Declarative object configuration

> Specify configuration files, __but do not specify the operation which should be performed__

`create`, `update`, `delete` operations are __automatically picked up__ by Kubernetes engine.

Pros of this approach:
- __We can easily work on directories of `config` files__ - all of them will be `applied` appropriately
- __Different configs allow us to handle more objects__
- __Only two commands needed__
- Does not replace the whole configuration "a-new", only patches it

To check which operations will be applied we can run:

In [ ]:
kubectl diff -f ./configs

# Recursively find all `.yaml` files

kubectl diff -R -f ./configs

Once we verify that parsed configs should bring cluster into the desired state one can run:

In [ ]:
kubectl apply -R -f ./configs # R for recusrive

Unfortunately:
- Might be harder to debug (especially when starting out with `k8s`)
- Complexity of operations hidden under the hood

# Workloads

> Workload is an application running on `kubernetes`

Each `workload` runs within __a set of `POD`s__, while `POD` __represents a set of containers running together__.

- `POD`s are scheduled to run on nodes
- In case of `node` failure all of the `POD`s on the node are also deleted (__`POD`s are ephemeral, should be easy to (re)create__)
- If `POD` run fails it will stay in inappropriate state (more details in `POD` section)

As there are a lot of failure points cluster admin would have to:
- Verify whether `POD`s are running fine constantly
- Manually reschedule `POD`s to different `Node`s
- Manually restart `POD`s in case any container failed

> __Manual handling is pretty inefficient, hence `Workload Resources` were created__

`Workload Resources` can specify (amongst other things):
- When `POD` should be recreated
- What to do in case of failures
- How many times should we try to reschedule before giving up 

> ## `Workload Resources` should be used to manage `POD`s lifecycles, AVOID DEPLOYING "BARE PODS"!

But before that we should know a little more about `POD`s themselves...

# PODs

> __A group of one or more containers, with shared context and a specification for how to run the containers__

PODs add another layer of abstraction over the containers and behave similarly to `docker-compose`, e.g. __they can connect multiple related containers into one logical grouping__.

Shared context:
- Shared storage
- Shared network resources (e.g. IP)
- Linux namespaces

__Individual applications might be further isolated within the `POD`__

> `POD` is minimal deployment unit in Kubernetes

This means we cannot schedule containers on their own.

## Lifecycle

> `POD`s remain on scheduled `Node` until termination (according to restart `policy` if failure occured) or deletion

__`POD`s are never moved across nodes, they are eventually recreated!__

Some features:
- `POD`s cannot self-heal (e.g. restsart themselves). This is done by appropriate `Workload Resources` or by cluster admin
- __PODs can restart failed containers though__ (using `kubelet`)
- Related resources (e.g. `volume`s) are also deleted after `POD` termination (unless specified otherwise)

`POD`s can be in one of multiple phases:
- `pending` - accepted by `k8s` cluster, but:
    - one or more containers didn't start
    - `POD` is waiting for node scheduling
    - container image is currently downloaded
- `running` - at least one `container` within `POD` is running (or being (re)started)
- `succeeded` - all containers in the `POD` have suceeded __and will not restart__
- `failed` - at least one container terminated in a failure
- `unknown` - state of `POD` ould not be obtained, __typically communication error with `Node`__

### Container states

> __Kubernetes also watches the state of individual containers within `POD`s__

Containers can be in one of three states:
- `Waiting` - downloading image or pulling `secrets`, __reason is also provided for monitoring__
- `Running`
- `Terminated` - either terminated successfully or not; __reason and `exit code` is provided for monitoring__

## Single Container POD

> Most common use case is POD containing a single container

In this case, we can think of a POD as a container wrapper

Examples could include:
- FastAPI server receiving requests and saving to shared database
- Docker container receiving images as requests and forwarding the classification

## Multiple Containers POD

> More advanced use case, multiple tightly coupled containers __making a cohesive unit of service__

![](./images/pod.svg)

Examples could include:
- Training multiple machine learning models, where:
    - First container accesses shared storage of raw data and transforms it
    - Second container trains neural network on the presented data
    - Third container pushes out the trained model to serving container
    - Fourth container serves the model
- One container serving data to the public (`read only` persmissions), while another, internal one, writes data to shared storage

> __POD containers are scheduled on the same "logical host" (for cloud, for clusters of servers: same VM or physical computer) due to tight coupling__

## Defining POD (POD template)

> `POD` as a basic `kind` can be specified via `.yaml` config file (__although specifying bare `POD`s is discouraged__)

Specifying "bare `POD`s" is pretty straightforward:

In [ ]:
---
apiVersion: v1
kind: Pod
metadata:
  name: pod1
  labels:
    tier: frontend
spec:
  containers:
  - name: hello1
    image: gcr.io/google-samples/hello-app:2.0

# Workload Resources

> Kubernetes provides quite a few `workload resources` out of the box

Before diving into specific `Workload Resources` let's get a few concepts straight:
- __Each `workload` presented below uses `.spec.template` field WHICH SPECIFIES HOW TO CREATE A `POD`__
- `template`is essneitally the same as `POD` config except `kind` and `apiVersion` (rest is performed the same)
- __Each `workload` has a `.spec.selector` WHICH SPECIFIES WHICH `POD`s ARE HANDLED BY THE `WORKLOAD RESOURCE`__
- `.spec.selector` use matches on defined `labels` __and may "take care of" `POD`s outside it's config file!__

With the above in mind, let's dive in:

## ReplicaSet

>  Maintains a stable set of replica Pods running at any given time

`ReplicaSet`:
- creates new `POD`s accordingly to `.spec.replicas` field value
    (from `.spec.template` config)
- deletes `POD`s if too many of them are scheduled to nodes

### Acquiring `POD`s

> `ReplicaSet` is linked to the `POD`s via `metadata.ownerReferences` and acquired via `.spec.selector` match

The above works something like this:
- Each pod has `metadata.ownerReferences` __automatically added by `k8s`__
- Above specifies __who manages the `POD`__ (e.g. another controller)
- __If `POD`__:
    - has no "owner" (e.g. bare `POD`) __or__
    - it's owner __is not another controller and__
    - `.spec.selector` fields match
- Then `POD` is acquired by the `ReplicaSet`

> __Above process works the same for other `workload resources` (or managers)__

### When should we use `ReplicaSet`s?

> __In general it is advised to use higher level `Deployment` `workload resource`__

`Deployment`, a higher level concept, __manages `ReplicaSet`s__ and in addition provides __declarative updates to `POD`s__

> Why is it still there?

Only reasons to go for `ReplicaSet`s would be:
- custom update orchestration
- we will never need to update `config`

Last one is pretty unlikely (and we are not paying large price for this feature), hence:

> ### USE `DEPLOYMENT` INSTEAD

(for those interested `ReplicaSet` is described in detail [here](https://kubernetes.io/docs/concepts/workloads/controllers/replicaset/))

## Deployment

> Provides declarative updates for `Pods` and `ReplicaSets`

Given about information, let's see whether we can tell what each field means in the config below:

In [ ]:
---
apiVersion: apps/v1
kind: Deployment
metadata:
  name: nginx-deployment
  labels:
    app: nginx
spec:
  replicas: 3
  selector:
    matchLabels:
      app: nginx
  template:
    metadata:
      labels:
        app: nginx
    spec:
      containers:
      - name: nginx
        image: nginx:1.14.2
        ports:
        - containerPort: 80

### Rollover

What happens when `Deployment` is updated?
- New `ReplicaSet` is created to bring up specified number of `POD`s
- `POD`s whose `.spec.selector` matches __but `.spec.template` is different from new `Deployment`__ are scaled down
- This process is performed until old deployment reaches `0` PODs and new one reaches `.spec.replicas` value

If, in this process, we update `Deployment` once again a `rollover` will happen:

> `rollover` means scaling down `ReplicaSet`s __as soon as new configuration comes in__

For example:
- First `Deployment` should create `5` PODs
- `Deployment` is updated while only `3` PODs where scheduled
- __`Deployment` will immediately scaled down `3` previous PODs AND WILL NOT WAIT FOR THE `5` TO COME UP!__

### Label selector updates

> __We SHOULD NOT update our `.spec.selector` and foresee how it should be designed__

`.spec.selector` is even immutable for `api/v1` of `kubernetes`!

To see possible drawbacks, check [this section](https://kubernetes.io/docs/concepts/workloads/controllers/deployment/#label-selector-updates).

> Keep `selector`s of different controllers separate at all times!

### Deployment status 

One of three states are generally available:
- `progressing` - __rolling out new `ReplicaSet` to match config__
- `complete` - desired state matches current
- `fail to progress`, possible factors:
    - Image pulling errors
    - Insufficient permissions
    - Insufficient resources on `Node`
    
For the last case we should specify `.spec.progressDeadlineSeconds=N` __which describes after how many seconds `Deployment` is considered stalled__

It should be noted that:
- __Above field DOES NOT END `Deployment` NOR `POD`!__
- Higher level orchestrators can use this information though to act accordingly, for example:
    - __`rollback` to previous `Deployment` settings__ (self-healing)
- This info will be added to `k8s` managed `.status` field

### Clean Up

We can also control how many old `ReplicaSets` should be preserved via `.spec.revisionHistoryLimit`. 

> This setting is set to `10` by default in order to support up to `10` rollbacks in case something goes wrong!

> __Setting this to `0` will erase possibility of `rollback`!__

> __This is only kept as `ReplicaSet` generated config, NO NODE WILL RUN OUR OLD `Deployment`!__

In general, leave at least value of `1`

## Writing `Deployment` spec

Below are a few useful information about `.yml` config schema of `Deployment`:

- `.spec.template.spec.restartPolicy=Always` is only allowed (e.g. `Deployment` will try to restart `POD`s indefinitely)
- `.spec.replicas=1` is the default value
- `.spec.selector` __MUST MATCH__ `.spec.template.metadata.` labels (__but can match more `POD`s hence you should look out__)
- `.spec.strategy="RollingUpdate"` is the default one (we saw description above):
    - We can specify `"Recreate"` __which will kill all old `ReplicaSet`s before creating new ones__
    - `.spec.strategy.rollingUpdate.maxUnavailable` - optional field; __percent, e.g. `=25%`__ of `POD`s which can be unavailable during update; __default: `25%`__
    - `.spec.strategy.rollingUpdate.maxSurge` - __maximum number of `POD`s over the desired value__ (specification as above); __default: `25%`__


Things to note:
- Using `.spec.strategy.maxSurge` we can control how resource hungry is our application
- Using `.spec.strategy.maxUnavailable` we can always ensure there are enough `POD`s to handle incoming traffics


> ### `Deployment` should be used for stateless applications

## DaemonSet

> __`DaemonSet` ensures that `Node`s run a copy of `POD`s as they are added to the cluster__

- When `Nodes` are removed `POD`s are also downscaled
- Deleting `DaemonSet` makes `k8s` garbage collect the `POD`s

### Usage

- Running cluster storage `daemon` on every `node`
- Running logs collection on every `node`
- Running `Node` monitoring on every `node`

> __Every long running `job` (daemon) per `Node` is a good fit for `DaemonSet`__

### Required fields

> Similar to `Deployment`, which means `.spec.template`, `.spec.selector` and __no `.spec.replicas` (as the same `daemon` is run per-node)__

- Acquisition of `POD`s happens on matchin `.spec.selector` (like previously)
- __Acquisition of `Node`s happens via `.spec.template.spec.nodeSelector` field__

### Assigning Pods to Nodes

> In general we don't have to interfere with `k8s` POD deployment to specific `Node`s

There might be a few reasons to do that though:
- Ensuring `POD` ends up on a `Node` which has `SSD` attached
- Co-locate `POD`s from different services in the same zone if they communicate frequently

`k8s` comes with a set of predefined `labels` for `Node`s, full list can be seen [here](https://kubernetes.io/docs/reference/labels-annotations-taints/), but to mention a few:
- region of deployment (in case of cloud): `topology.kubernetes.io/region=us-east-1`
- ip address of a node: `kubernetes.io/hostname=ip-172-20-114-199.ec2.internal`
- operating system our node is running: `kubernetes.io/os=linux`

> __During `kubectl` lesson we will learn how to add our own `label`s to `Node`s__

Given the above, we can use the `.spec.template.spec.nodeSelector` as below:

In [ ]:
---
apiVersion: apps/v1
kind: DaemonSet
metadata:
  name: fluentd-elasticsearch
spec:
  selector:
    matchLabels:
      name: fluentd-elasticsearch
  template:
    metadata:
      labels:
        name: fluentd-elasticsearch
    spec:
      containers:
      - name: fluentd-elasticsearch
        image: quay.io/fluentd_elasticsearch/fluentd:v2.5.2
        resources:
          limits:
            memory: 200Mi
          requests:
            cpu: 100m
            memory: 200Mi
      terminationGracePeriodSeconds: 30
      nodeSelector:
        kubernetes.io/os!=linux

### Updating `DaemonSet`

- __Intuitive per-node matching__: if we change `label`s `daemon`s will be added to newly matching __and removed from old `Node`s__
- Deleting `DaemonSet` can be performed using `kubectl` normally (as we will see in the upcoming notebooks)
- `POD`s are replaced according to `updateStrategy` mentioned in `Deployment`

### Why `DaemonSet`s?

One could also use `init` scripts on the nodes (even set up automatically during OS installation via [`packer`](https://www.packer.io/)), but:
- It would be hard to monitor running daemons on multiple clusters
- Different configs for `daemons` and `k8s` applications
- Isolation between `daemons` and `apps` (although could be done otherwise)
- Easier to create more complicated `daemon`s

Versus deploying "Bare `POD`s":
- `DaemonSet` replaces failed `daemons` accordingly to `restartStrategy`

Versus `Deployment`:
- Handles similiar loads (__jobs which are not supposed to terminate, e.g. `server`__)
- `DaemonSet` allows us to choose specific `Node`s we would like the service to run on
- `DaemonSet` does not need `replica`s as it works on a per-node basis

## Jobs

> Creates one or more `POD`s and will continue to retry execution of the `POD`s until a specified number of them successfully terminate

Use cases:
- create one `Job` to make sure our task runs successfully
- run the same `Job` in parallel for `N` times

An example config `Job` workload calculating `pi` value could be:

In [ ]:
apiVersion: batch/v1
kind: Job
metadata:
  name: pi
spec:
  template:
    spec:
      containers:
      - name: pi
        image: perl
        command: ["perl",  "-Mbignum=bpi", "-wle", "print bpi(2000)"]
      restartPolicy: Never
  backoffLimit: 4

### `.spec`ification

Standard fields are necessary, but in addition:

- `.spec.restartPolicy` can be either `Never` or `OnFailure` (default)
- `.spec.completions` - how many `Job`s have to finish
- `.spec.parallelism` - how many `POD`s with our job should be scheduled at the same time

Using `.spec.completions` and `.spec.parallelism` we can construct different levels of parallelism:

- __non-parallel__ - specify `.spec.completions=1`, only one `Job` will be created. __New one will only start after this one fails!__
- __parallel with fixed completion count__ - specify `.spec.completions=N` to run __at most `N` parallel jobs at a given time__ (controller will reschedule `Node`s in case of failure)
- __parallel with work queue__ - `.spec.completions=1` and `.spec.parallelism=N` - `N` PODs will run, after first one succeeds __the rest will continue execution until termination__ (for improved efficiency we need to implement direct `POD` to `POD` communication)

`non-parallel` is the default mode as `.spec.completions=1` and `.spec.parallelism=1` are the default values.

One could also specify [Completion Mode](https://kubernetes.io/docs/concepts/workloads/controllers/job/#completion-mode) which allows us to modify their behaviour upon termination.

- `.spec.backoffLimit` - how many times __a single `POD`__ should be restarted before considering `Job` failed

In this case:
- Depending on the settings, but if `.spec.completions=N` is hit, __job succeeded__
- Until this moment, try to recreate `POD` with `Job` `N` times
- Exponential back-off delay is applied, e.g.:
    - First retry after `10s`
    - Second after `20s`
    - Third after `40s`
    - __Capped at `6m` backoff!__
    
- `.spec.activeDeadlineSeconds` - how many seconds __for the whole job__ until termination. Once reached, __all of the `POD`s ater terminated__ (takes precedence over `.spec.backoffLimit`)
    
### Cleanup

> `Job` after finishing __will not be automatically removed from the cluster__

Why is that bad?

> `kube-apiserver` will still query `Job` and look for it's `POD`s __putting unneeded pressure on `k8s`__

Why this is the default behaviour?
- One might want to check logs of finished jobs (which are stored within `POD` or external storage)
- Checking status of `Job`(s)

What one can do is to set up a so called __`TTL` (time to live)__:

> __`TTL` specifies after how long should the `job` be removed from the cluster (including all of it's `POD`s and other dependencies)__.

One could do that via `.spec.ttlSecondsAfterFinished` field as one can read below:

In [ ]:
apiVersion: batch/v1
kind: Job
metadata:
  name: pi-with-ttl
spec:
  ttlSecondsAfterFinished: 100
  template:
    spec:
      containers:
      - name: pi
        image: perl
        command: ["perl",  "-Mbignum=bpi", "-wle", "print bpi(2000)"]
      restartPolicy: Never

# Challenges

> ### You should try these challenges after finishing all of the `kubernetes` related lessons!

It will make your progress substantially faster.


## Mandatory

- What are [addons](https://kubernetes.io/docs/concepts/overview/components/#addons) in Kubernetes ecosystem? Which one is necessary?
- Check out [kustomize](https://kubernetes.io/blog/2018/05/29/introducing-kustomize-template-free-configuration-customization-for-kubernetes/) (available via `kubectl`). What are the use cases for it? 
- See additional ways to match `POD`s to specific `Node`s [here](https://kubernetes.io/docs/concepts/scheduling-eviction/assign-pod-node/)
- Check out [`CronJob` workload resource](https://kubernetes.io/docs/concepts/workloads/controllers/cron-jobs/). Make sure you understand the concept and `cron` syntax included.

## Additional

- Check out [Kubernetes API Conventions](https://github.com/kubernetes/community/blob/master/contributors/devel/sig-architecture/api-conventions.md) to get deeper understanding of Kubernetes Objects and internals of the project.
- What are the alternatives to `minikube`? What is `kind` or `k3s`? What are upsides/downsides of using them?
- Check out [Client Libraries](https://kubernetes.io/docs/reference/using-api/client-libraries/) (which allow us to communicate with `kubelet-apiserver` via certain programming languages). Check out [Python Client Library](https://github.com/kubernetes-client/python/). Which task could you automate using it?
- Check out [imperative commands way to manage `k8s` objects](https://kubernetes.io/docs/concepts/overview/working-with-objects/object-management/#imperative-commands), __but don't use it!__ What are the downsides due to which we did not describe it?
- Read more about [TTL (time to live) Controller](https://kubernetes.io/docs/concepts/workloads/controllers/ttlafterfinished/).
- Read about [canary `Deployment`s](https://kubernetes.io/docs/concepts/cluster-administration/manage-deployment/#canary-deployments)
- Read about [`Job` template expansion](https://kubernetes.io/docs/tasks/job/parallel-processing-expansion/)
- What are `DaemonSet` tolerations?